In [2]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd

In [3]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")
dataset.head(10)

,index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
5,5,03/05/1915,01:00,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,NaN,41.0,21.0,0.0,Crashed into trees while attempting to land af...
6,6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,NaN,NaN,Zeppelin L-10 (airship),NaN,NaN,19.0,19.0,0.0,"Exploded and burned near Neuwerk Island, when..."
7,7,07/28/1916,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,20.0,0.0,"Crashed near the Black Sea, cause unknown."
8,8,09/24/1916,01:00,"Billericay, England",Military - German Navy,NaN,NaN,Zeppelin L-32 (airship),NaN,NaN,22.0,22.0,0.0,Shot down by British aircraft crashing in flames.
9,9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,NaN,NaN,Zeppelin L-31 (airship),NaN,NaN,19.0,19.0,0.0,Shot down in flames by the British 39th Home D...


In [32]:
db_name='airplane_crashes'
try:
    mydb = mysql.connect(host='localhost', user='root', password='franci22') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")

        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

[('airplane_crashes',), ('electronics',), ('GoShare',), ('information_schema',), ('mid_practice',), ('movies',), ('mysql',), ('parents',), ('performance_schema',), ('sys',)]
The database already exists! The old database has been deleted!)
Database is created


In [33]:
mycursor.execute('USE' + ' ' + db_name)

In [34]:
#create tables
mycursor.execute(
      '''
        CREATE TABLE `Airplane` (
          `serial_number` varchar(100) UNIQUE ,
          `registration` varchar(100) UNIQUE ,
          `operator` varchar(100),
          `type` varchar(100),
          PRIMARY KEY (`serial_number`, `registration`)
        );
      '''
      )

In [35]:
mycursor.execute(
      '''
        CREATE TABLE `Flight` (
           `flight_number` varchar(100) UNIQUE ,
           `aboard` int,
           `route` varchar(100),
           `serial_number` varchar(100),
           `registration` varchar(100),
           PRIMARY KEY (`flight_number`),
           foreign key (`registration`, `serial_number`) REFERENCES `Airplane` (`registration`,`serial_number`)
        );
      '''
      )

In [36]:
mycursor.execute(
    '''
    CREATE TABLE `Crash` (
  `index` int,
  `location` varchar(100),
  `time` time,
  `date` date,
  `summary` varchar(100),
  `serial number` varchar(100),
  `registration` varchar(100),
  PRIMARY KEY (`index`),
  FOREIGN KEY (`serial number`) REFERENCES `Airplane`(`serial_number`),
  FOREIGN KEY (`registration`) REFERENCES `Airplane`(`registration`)
);
    '''
)

In [37]:
mycursor.execute(
    '''
    CREATE TABLE `Fatalities` (
    `number of fatalities` int,
    `ground` int,
    `crash index` int,
    PRIMARY KEY (`crash index`),
    FOREIGN KEY (`crash index`) REFERENCES `Crash`(`index`)
);
    '''
)

In [ ]:
d